In [2]:
import pandas as pd
import sys
from pyspark.sql import types as T
from pyspark.sql import functions as F

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()
spark

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Importamos los datos de entrenamiento y los de prueba
ruta="C:/Users/Aitor/Desktop/TFM/datos_mod_cab/"
fichero_entrenamiento = "final_dataset.csv"

In [7]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df_train = pd.read_csv(ruta+fichero_entrenamiento, header=0)
#df_train = df_train.drop(columns=['time(us)','accX(g)','accY(g)','accZ(g)','gyrX(o/s)','gyry(o/s)'])
df_train = df_train.drop(columns=['user_ID','attemp','time(us)'])
# Print first 10 samples
df_train.describe()

,accX(g),accY(g),accZ(g),gyrX(o/s),gyry(o/s),gyrz(o/s)
count,1.405300e+06,1.405300e+06,1.405300e+06,1.405300e+06,1.405300e+06,1.405300e+06
mean,9.675167e-01,-1.702909e-01,1.718306e-02,8.716722e-01,4.921452e-02,-7.559592e-01
std,5.337054e-01,2.643577e-01,2.076064e-01,4.270605e+00,1.050591e+01,6.357781e+01
min,-1.593848e+01,-1.027734e+01,-1.501758e+01,-3.602290e+02,-4.930992e+02,-4.528092e+02
25%,8.828120e-01,-4.663090e-01,-3.418000e-03,5.496180e-01,-2.290080e-01,-1.175573e+00
50%,1.005859e+00,-2.246100e-02,1.611300e-02,8.702290e-01,4.580200e-02,-7.633590e-01
75%,1.014648e+00,6.348000e-03,3.564500e-02,1.114504e+00,2.900760e-01,-5.343510e-01
max,1.599951e+01,8.896484e+00,1.548096e+01,1.594504e+02,4.930534e+02,4.970076e+02


In [23]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df_train = pd.read_csv(ruta+fichero_entrenamiento, header=0)
df_train = df_train.drop(columns=['time(us)'])
# Print first 10 samples
print(df_train)

KeyError: "['time(us)'] not found in axis"

In [42]:
# Divide data into features X and target (Classes) Y
X_train = df_train.iloc[:,0:3]
Y_train = df_train.iloc[:,0]
print(X_train, Y_train)

         user_ID  attemp  gyrz(o/s)
0              1       1  -0.778626
1              1       1  -0.580153
2              1       1  -0.625954
3              1       1  -0.625954
4              1       1  -0.916031
...          ...     ...        ...
1405295       47      20  -0.549618
1405296       47      20  -0.595420
1405297       47      20  -0.824427
1405298       47      20  -0.671756
1405299       47      20  -0.488550

[1405300 rows x 3 columns] 0           1
1           1
2           1
3           1
4           1
           ..
1405295    47
1405296    47
1405297    47
1405298    47
1405299    47
Name: user_ID, Length: 1405300, dtype: int64


In [23]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
print(X_train)

          0    1         2         3
0       0.0  0.0  0.515307  0.412609
1       0.0  0.0  0.515291  0.412844
2       0.0  0.0  0.515196  0.412790
3       0.0  0.0  0.515465  0.412790
4       0.0  0.0  0.515481  0.412447
...     ...  ...       ...       ...
351320  1.0  1.0  0.515402  0.413060
351321  1.0  1.0  0.515244  0.412627
351322  1.0  1.0  0.515433  0.412736
351323  1.0  1.0  0.514944  0.412573
351324  1.0  1.0  0.514912  0.412934

[351325 rows x 4 columns]


In [24]:
# Convert target Y to one hot encoded Y for Neural Network
Y_train = pd.get_dummies(Y_train)
# If target is in string form, use following code:
# First encode target values as integers from string
# Then perform one hot encoding
# encoder = LabelEncoder()
# encoder.fit(Y)
# Y = encoder.transform(Y)
# Y = np_utils.to_categorical(Y)
print(Y_train)

        1   2   3   4   5   6   7   8   9   10  ...  38  39  40  41  42  43  \
0        1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
1        1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
2        1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
3        1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
4        1   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..   
351320   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
351321   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
351322   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
351323   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   
351324   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   

        44  45  46  47  
0        0   0   0   0  
1

In [25]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X_train = X_train.values
Y_train = Y_train.values

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report,precision_recall_fscore_support,accuracy_score

In [44]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
precisionScores = []
f1Scores = []
recallScores = []
for train, test in kfold.split(X_train,Y_train):
    X_trained = pd.DataFrame(X_train)
    Y_trained = pd.get_dummies(Y_train)
    X_training = X_trained.values
    Y_training = Y_trained.values
  # Create model here
    model = Sequential()
    model.add(Dense(64, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(47, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    # Fit the model
    model.fit(X_training[train], Y_training[train], epochs=8, batch_size=512, verbose=0)
    # evaluate the model
    scores = model.evaluate(X_training[test], Y_training[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    #matriz confusion
    y_pred=model.predict(X_training[test]) 
    y_pred=np.argmax(y_pred, axis=1)
    y_test=np.argmax(Y_training[test], axis=1)
    cm = confusion_matrix(y_test, y_pred)
    #print(cm)
    scores2 = precision_recall_fscore_support(y_test, y_pred,average='macro',zero_division='warn')
    print(precision_recall_fscore_support(y_test, y_pred,average='macro',warn_for ='recall',zero_division='warn'))
    precisionScores.append(scores2[0] * 100)
    f1Scores.append(scores2[1] * 100)
    recallScores.append(scores2[2] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("precision: %.2f%% ; f1: %.2f%% ; recall: %.2f%%" % (np.mean(precisionScores),np.mean(f1Scores),np.mean(recallScores)))

accuracy: 100.00%
(1.0, 1.0, 1.0, None)
(1.0, 1.0, 1.0, None)
accuracy: 100.00%
(1.0, 1.0, 1.0, None)
(1.0, 1.0, 1.0, None)
accuracy: 100.00%
(0.9999928864608008, 0.9999928840816907, 0.9999928840814917, None)
(0.9999928864608008, 0.9999928840816907, 0.9999928840814918, None)
accuracy: 100.00%
(1.0, 1.0, 1.0, None)
(1.0, 1.0, 1.0, None)
accuracy: 99.96%
(0.9995910816739491, 0.999587276738063, 0.9995872696217447, None)
(0.9995910816739493, 0.9995872767380631, 0.9995872696217446, None)
99.99% (+/- 0.02%)
precision: 99.99% ; f1: 99.99% ; recall: 99.99%
